In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext  
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils   

sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, 5)

ks_click = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'ec2-52-9-120-149.us-west-1.compute.amazonaws.com:9092'})
line_click = ks_click.map(lambda x: x[1])
import re
def mk_int(s):
    s = s.strip()
    return int(s) if s else 0
def tuple_sum(a, b):
    return [a[0]+b[0], a[1]+b[1]]

ctr = line_click.flatMap(lambda line: [line.split("\t")[-1]]) \
    .map(lambda click: ("ctr", [mk_int(click), 1])) \
    .reduceByKey(lambda a, b: tuple_sum(a, b)) \
    .flatMap(lambda row: ("ctr = ", 100.0 * row[1][0] / row[1][1]))
    
ctr.pprint()
ssc.start()  
ssc.awaitTermination()
ssc.stop()
sc.stop()